FEATURE SELECTION

In [55]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os

In [56]:
# import package scikit-learn 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt

In [57]:
os.getcwd()

'C:\\Users\\eddym\\Desktop\\Kingas\\School\\TMet University\\CIND820 - Big Data Analytics Project\\Project'

In [58]:
#Import the data --> this file represents only the entire  set
data = pd.read_csv("Train_set_CC.csv", encoding = "latin-1")
del data["Unnamed: 0"]
data.head()

,FV_D,FV_M,FV_DATE,VISIT_CODE,WP_ID,NAICS,RegCode,INDCode,HICCode,HICPerCode,E1524Code,E2554Code,E55pCode,ConCode,Claims_Code
0,29,6,2019-06-29,2019-06-29 2093649,2093649,812115,3,10,1,2,1,1,2,1,1
1,25,2,2019-02-25,2019-02-25 1095872,1095872,812330,2,10,2,2,1,1,2,4,1
2,19,6,2019-06-19,2019-06-19 2091294,2091294,812190,2,10,2,2,1,1,2,4,1
3,6,5,2019-05-06,2019-05-06 2029237,2029237,238990,4,4,5,5,2,2,2,1,3
4,25,10,2019-10-25,2019-10-25 2125435,2125435,236110,4,4,5,5,2,2,2,3,3


In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19070 entries, 0 to 19069
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   FV_D         19070 non-null  int64 
 1   FV_M         19070 non-null  int64 
 2   FV_DATE      19070 non-null  object
 3   VISIT_CODE   19070 non-null  object
 4   WP_ID        19070 non-null  int64 
 5   NAICS        19070 non-null  int64 
 6   RegCode      19070 non-null  int64 
 7   INDCode      19070 non-null  int64 
 8   HICCode      19070 non-null  int64 
 9   HICPerCode   19070 non-null  int64 
 10  E1524Code    19070 non-null  int64 
 11  E2554Code    19070 non-null  int64 
 12  E55pCode     19070 non-null  int64 
 13  ConCode      19070 non-null  int64 
 14  Claims_Code  19070 non-null  int64 
dtypes: int64(13), object(2)
memory usage: 2.2+ MB


In [60]:
list(data.columns)

['FV_D',
 'FV_M',
 'FV_DATE',
 'VISIT_CODE',
 'WP_ID',
 'NAICS',
 'RegCode',
 'INDCode',
 'HICCode',
 'HICPerCode',
 'E1524Code',
 'E2554Code',
 'E55pCode',
 'ConCode',
 'Claims_Code']

In [61]:
data.corr()

,FV_D,FV_M,WP_ID,NAICS,RegCode,INDCode,HICCode,HICPerCode,E1524Code,E2554Code,E55pCode,ConCode,Claims_Code
FV_D,1.000000,-0.005572,0.009467,0.009305,-0.011435,0.006454,0.003082,0.001565,0.002417,0.004056,-0.004609,0.004584,0.002610
FV_M,-0.005572,1.000000,0.055847,-0.032057,0.000191,-0.035038,-0.025402,0.045479,0.048980,0.052007,0.042648,-0.047990,-0.031168
WP_ID,0.009467,0.055847,1.000000,-0.207211,-0.132408,-0.193444,0.070666,0.421822,0.438143,0.290313,0.413452,0.025102,-0.034452
NAICS,0.009305,-0.032057,-0.207211,1.000000,0.004190,0.794010,-0.225583,-0.635153,-0.462673,-0.698301,-0.496228,-0.042166,-0.122529
RegCode,-0.011435,0.000191,-0.132408,0.004190,1.000000,-0.000130,-0.252222,-0.057832,-0.054290,-0.008155,-0.097321,-0.080561,-0.223848
INDCode,0.006454,-0.035038,-0.193444,0.794010,-0.000130,1.000000,0.132269,-0.768598,-0.120758,-0.800506,-0.685117,-0.045927,0.238653
HICCode,0.003082,-0.025402,0.070666,-0.225583,-0.252222,0.132269,1.000000,-0.024012,0.261644,-0.056406,-0.007347,0.014014,0.953870
HICPerCode,0.001565,0.045479,0.421822,-0.635153,-0.057832,-0.768598,-0.024012,1.000000,0.472012,0.772725,0.681068,0.047431,-0.242506
E1524Code,0.002417,0.048980,0.438143,-0.462673,-0.054290,-0.120758,0.261644,0.472012,1.000000,0.371712,0.130929,0.039491,0.141153
E2554Code,0.004056,0.052007,0.290313,-0.698301,-0.008155,-0.800506,-0.056406,0.772725,0.371712,1.000000,0.482890,0.007927,-0.186687


In [62]:
# define the data (X) predictors and (y) outcome  

X = data.drop(['FV_DATE','VISIT_CODE', 'Claims_Code'], axis=1)
y = data['Claims_Code']

#split into train and test sets 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=23)
X.head()

,FV_D,FV_M,WP_ID,NAICS,RegCode,INDCode,HICCode,HICPerCode,E1524Code,E2554Code,E55pCode,ConCode
0,29,6,2093649,812115,3,10,1,2,1,1,2,1
1,25,2,1095872,812330,2,10,2,2,1,1,2,4
2,19,6,2091294,812190,2,10,2,2,1,1,2,4
3,6,5,2029237,238990,4,4,5,5,2,2,2,1
4,25,10,2125435,236110,4,4,5,5,2,2,2,3


In [63]:
#Univariate chi^2 feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


#function to select best features using chi^2
def best_feat(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k="all")
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs



In [64]:
#run the function
X_tr_feat, X_ts_feat, fs = best_feat(X_train, y_train, X_test)

for i in range(len(fs.scores_)):
    print('Feature %d: %.2f' % (i, fs.scores_[i]))

Feature 0: 5.91
Feature 1: 112.72
Feature 2: 320083450.62
Feature 3: 400382299.00
Feature 4: 2786.29
Feature 5: 9448.07
Feature 6: 11131.01
Feature 7: 4709.08
Feature 8: 230.16
Feature 9: 917.74
Feature 10: 511.51
Feature 11: 80.87


In [65]:
#4 top important features as per chi^2 approach--> 
X.iloc[:,[2,3,6,5,7]]

,WP_ID,NAICS,HICCode,INDCode,HICPerCode
0,2093649,812115,1,10,2
1,1095872,812330,2,10,2
2,2091294,812190,2,10,2
3,2029237,238990,5,4,5
4,2125435,236110,5,4,5
...,...,...,...,...,...
19065,2098918,236220,6,4,4
19066,2121494,237130,6,4,4
19067,366322,441120,3,12,1
19068,2060798,442210,7,12,1


In [35]:
# define the data (X) predictors and (y) outcome

X = data.drop(['FV_DATE','VISIT_CODE', 'Claims_Code'], axis=1)
y = data['Claims_Code']

#split into train and test sets 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=23)



In [66]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline

In [67]:
#RFE method for identifying most important features
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
rfe.fit(X_train, y_train)


RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)

In [68]:
#Identify most important features using RFE method

from operator import itemgetter
Imp_feat = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , Imp_feat), key=itemgetter(0))):
    print(x, y)

1 NAICS
1 RegCode
1 HICCode
1 E1524Code
1 E2554Code
2 HICPerCode
3 ConCode
4 E55pCode
5 INDCode
6 WP_ID
7 FV_M
8 FV_D
